In [1]:
import numpy as np
import pandas as pd
from numba import jit
import matplotlib.pyplot as plt

import pickle
import time
from ipypb import track

from general_functions1 import sqrt_err_relative, check_coo_tensor, gen_coo_tensor
from general_functions1 import create_filter, hr

import sys
sys.path.append("/notebook/Relations_Learning/") 
import CP_ALS3.CP_ALS3 as cp

In [2]:
!pwd

/notebook/Relations_Learning/gpu


In [3]:
with open('/notebook/Relations_Learning/test_filter.pkl', 'rb') as f:
    test_filter = pickle.load(f)
    
with open('/notebook/Relations_Learning/valid_filter.pkl', 'rb') as f:
    valid_filter = pickle.load(f)

In [4]:
from t_alg import mttcrp, mttcrp1, get_elem_deriv_tensor, factors_to_tensor, gcp_grad, multi_ind_to_indices, indices_to_multi_ind

from samplings import give_ns, generate_data

from elementwise_grads import bernoulli_logit_loss, bernoulli_logit_loss_grad

In [5]:
len(valid_filter)

17535

In [6]:
import torch
from torch import optim
device=torch.device("cuda:4")

class GSP_SGD(torch.nn.Module):
    def __init__(self, num_ent, num_rel, dim_emb, shape, loss_function, loss_function_grad):
        super(GSP_SGD, self).__init__()
                
        self.ent = torch.empty((num_ent, dim_emb), device = device)
        xavier_normal_(a_torch)
        a_torch.requires_grad=True

        self.rel = torch.empty((num_rel, dim_emb), device = device)
        xavier_normal_(b_torch)
        b_torch.requires_grad=True
        
        self.optimizer = optim.Adam([a_torch, b_torch, c_torch], lr=1e-3)
        
        self.coo = coo_tensor
        self.vals = vals
        self.shape = shape
        self.elemwise_loss = loss_function
        self.elemwise_grad = loss_function_grad

        #c_torch = torch.empty((num_ent, dim_emb), device = device)
        #xavier_normal_(c_torch)
        #c_torch.requires_grad=True
    
    def forward(self, ent_idx, rel_idx):
        pass

        

In [7]:
import numpy as np

def gcp_grad(coo, val, shape, a, b, l2, loss_function, loss_function_grad, device):
    """
        GCP loss function and gradient calculation.
        All the tensors have the same coordinate set: coo_tensor.
    """

    # Construct sparse kruskal tensor
    kruskal_val = torch.sum((a[coo[:,0], :] * b[coo[:,1], :] * a[coo[:,2], :]),1)
    #factors_to_tensor(coo_tensor, vals, a, b, c)
    
    # Calculate mean loss on known entries
    loss = loss_function(val, kruskal_val)
    # Compute the elementwise derivative tensor
    deriv_tensor_val = loss_function_grad(val, kruskal_val)
    
    #print ("in qcp_grad in deriv_tensor_val ", deriv_tensor_val)
    # Calculate gradients w.r.t. a, b, c factor matrices
    g_a = mttcrp1(coo, deriv_tensor_val, shape, 0, b, a, device)
    g_b = mttcrp1(coo, deriv_tensor_val, shape, 1, a, a, device)
    g_c = mttcrp1(coo, deriv_tensor_val, shape, 2, a, b, device)
    
    #print ("\n\n")
    
    
    # Add L2 regularization
    if l2 != 0:
        g_a += l2 * a[coo[0], :]
        g_b += l2 * b[coo[1], :]
        g_c += l2 * c[coo[2], :]
    
    return loss, g_a, g_b, g_c

In [8]:
#np.save('/notebook/Relations_Learning/a200.npz', a)
#np.save('/notebook/Relations_Learning/b200.npz', b)
#np.save('/notebook/Relations_Learning/c200.npz', c)

In [9]:
a.shape

NameError: name 'a' is not defined

In [39]:
a = np.load('/notebook/Relations_Learning/results/gpu_a.npz')
b = np.load('/notebook/Relations_Learning/results/gpu_b.npz')
c = np.load('/notebook/Relations_Learning/results/gpu_c.npz')

FileNotFoundError: [Errno 2] No such file or directory: '/notebook/Relations_Learning/results/gpu_a.npz'

In [10]:
path_data = "/notebook/Relations_Learning/Link_Prediction_Data/FB15K237/"
entity_list = pickle.load(open(path_data + 'entity_list', 'rb'))
relation_list = pickle.load(open(path_data + 'relation_list', 'rb'))

train_triples = pickle.load(open(path_data + 'train_triples', 'rb'))
valid_triples = pickle.load(open(path_data + 'valid_triples', 'rb'))
test_triples = pickle.load(open(path_data + 'test_triples', 'rb'))
train_valid_triples = pickle.load(open(path_data + 'train_valid_triples', 'rb'))

entity_map = pickle.load(open(path_data + 'entity_map', 'rb'))
relation_map = pickle.load(open(path_data + 'relation_map', 'rb'))

all_triples = train_valid_triples + test_triples

In [11]:
len(train_triples)

272115

In [12]:
#test_filter = create_filter(test_triples, all_triples)
#train_filter = create_filter(train_triples, all_triples)  
#valid_filter = create_filter(valid_triples, all_triples)  

In [13]:
#with open('train_filter.pkl', 'wb') as handle:
    #pickle.dump(train_filter, handle)

In [14]:
with open('/notebook/Relations_Learning/test_filter.pkl', 'rb') as f:
    test_filter = pickle.load(f)
    
with open('/notebook/Relations_Learning/valid_filter.pkl', 'rb') as f:
    valid_filter = pickle.load(f)

In [18]:
values = [1] * len(train_triples)
values = np.array(values, dtype=np.int64)

coords = np.array(train_triples, dtype=np.int64)
nnz = len(train_triples)
data_shape = (len(entity_list), len(relation_list), len(entity_list))
data_shape

(14541, 237, 14541)

In [16]:
num_epoch = 1000 
rank = 100 
l2 =  0
lr = 1e-2 
seed = 13 
hm = 1000
how_many = 2

In [22]:
!nvidia-smi

Thu Jun 17 12:03:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   30C    P8    13W / 260W |   1968MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 27%   

In [19]:
err_list

NameError: name 'err_list' is not defined

In [23]:
from torch.optim.lr_scheduler import StepLR
from timeit import default_timer as timer

coo_tensor = coords
vals = values
shape = data_shape
loss_function = bernoulli_logit_loss
loss_function_grad = bernoulli_logit_loss_grad

from torch.nn.init import xavier_normal_
from torch import optim

device=torch.device("cuda:1")

num_epoch = 12

random_state = np.random.seed(seed)

batch_size = 56
init_mind_set = set(indices_to_multi_ind(coo_tensor, shape))
coo_ns = np.empty((how_many * len(init_mind_set) + vals.size, 3), dtype=np.int64)
vals_ns = np.empty((how_many * len(init_mind_set) + vals.size,), dtype=np.float64)

err_arr = np.empty((num_epoch*vals_ns.shape[0]//batch_size + 1, ), dtype=np.float64)
error = 0.0
it = 0

num_ent = 14541
dim_emb = 200
num_rel = 237

a_torch = torch.empty((num_ent, dim_emb), requires_grad = True, device = device)
xavier_normal_(a_torch)
a_torch.grad = torch.zeros(a_torch.shape, device = device)

b_torch = torch.empty((num_rel, dim_emb), requires_grad = True, device = device)
xavier_normal_(b_torch)
b_torch.grad = torch.zeros(b_torch.shape, device = device)

optimizer = optim.Adam([a_torch, b_torch], lr=1e-3)
scheduler = StepLR(optimizer, step_size=2, gamma=0.5)

show_iter = True

start = timer()
for epoch in range(num_epoch):
    coo_ns, vals_ns = generate_data(coo_tensor, vals, init_mind_set, shape, how_many, epoch)
    coo_ns = torch.tensor(coo_ns, device = device)
    vals_ns = torch.tensor(vals_ns, device = device)
    shuffler = np.random.permutation(vals_ns.shape[0])
    coo_ns = coo_ns[shuffler]
    vals_ns = vals_ns[shuffler]
    #idxs = np.random.permutation(vals_ns.shape[0])
    print (vals_ns.shape[0], batch_size, vals_ns.shape[0]//batch_size)
    err_list = []
    for i in range(vals_ns.shape[0]//batch_size):
        # Get loss and gradients per sample
        # print ("coo_ns[i], vals_ns[i]", coo_ns[i], vals_ns[i])
        end = min(vals_ns.shape[0] - 1, (i+1)*batch_size)
        loss, g_a, g_b, g_c = gcp_grad(
            coo_ns[i*batch_size : end], vals_ns[i*batch_size : end], shape, a_torch, b_torch,
            l2, loss_function, loss_function_grad, device
        )
        err_list.append(loss.cpu().detach().numpy().mean())

        a_elems = coo_ns[i*batch_size : end, 0]
        b_elems = coo_ns[i*batch_size : end, 1]
        c_elems = coo_ns[i*batch_size : end, 2]
        
        a_torch.grad[a_elems, :] = g_a
        b_torch.grad[b_elems, :] = g_b
        a_torch.grad[c_elems, :] = g_c
        optimizer.step()
        a_torch.grad = torch.zeros(a_torch.shape, device = device)
        b_torch.grad = torch.zeros(b_torch.shape, device = device)
        
        err_arr[it] = np.mean(err_list)
        if show_iter and i%500 == 0:
            print("Iter: ", it, "; Error: ", np.mean(np.array(err_list)))
        it += 1   
    scheduler.step()
    a = a_torch.cpu().data.numpy()
    b = b_torch.cpu().data.numpy()
    c = a_torch.cpu().data.numpy()
    print ("count hr")
    print (hr(valid_filter[:1000], valid_triples[:1000], a, b, c, [1, 3, 10]) )
    end = timer()
    print (end - start)
    np.save('/notebook/Relations_Learning/results/gpu_a.npz', a_torch.cpu().data.numpy())
    np.save('/notebook/Relations_Learning/results/gpu_b.npz', b_torch.cpu().data.numpy())
    np.save('/notebook/Relations_Learning/results/gpu_c.npz', a_torch.cpu().data.numpy())
    

816345 56 14577
Iter:  0 ; Error:  0.6931426404652744
Iter:  500 ; Error:  0.6931448181376691
Iter:  1000 ; Error:  0.6931259831014395
Iter:  1500 ; Error:  0.6930076131455367
Iter:  2000 ; Error:  0.6924960714221322
Iter:  2500 ; Error:  0.6911811214942308
Iter:  3000 ; Error:  0.6886549989208472
Iter:  3500 ; Error:  0.6851194491714021
Iter:  4000 ; Error:  0.6804785100865407
Iter:  4500 ; Error:  0.6751001314411018
Iter:  5000 ; Error:  0.6688967129989343
Iter:  5500 ; Error:  0.6623912733726212
Iter:  6000 ; Error:  0.6557488714034669
Iter:  6500 ; Error:  0.6489698550831051
Iter:  7000 ; Error:  0.6422769228120526
Iter:  7500 ; Error:  0.6357087923589181
Iter:  8000 ; Error:  0.6290571567597771
Iter:  8500 ; Error:  0.6224255486734745
Iter:  9000 ; Error:  0.6156572949817392
Iter:  9500 ; Error:  0.6086810934407346
Iter:  10000 ; Error:  0.6012435596833925
Iter:  10500 ; Error:  0.5932137194358816
Iter:  11000 ; Error:  0.5841045068556182
Iter:  11500 ; Error:  0.5741783603679865


KeyboardInterrupt: 

In [24]:
%%time
hr(valid_filter[:1000], valid_triples[:1000], a, b, c, [1, 3, 10])  #[:1000]

CPU times: user 7.1 s, sys: 160 ms, total: 7.26 s
Wall time: 7.57 s


(0.21, 0.316, 0.47, 0.2932382942815133)

In [ ]:
%%time
hr(valid_filter, valid_triples, a, b, c, [1, 3, 10])  #[:1000]

In [ ]:
%%time
hr(test_filter, test_triples, a, b, c, [1, 3, 10])

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable, Function
import random

import math

class WARP(Function): 
    '''
    autograd function of WARP loss
    '''
    @staticmethod
    def forward(input, target, max_num_trials = None):
        
        batch_size = target.size()[0]
        
        if max_num_trials is None: 
            max_num_trials = target.size()[1] - 1
        
        positive_indices = torch.zeros(input.size())
        negative_indices = torch.zeros(input.size())
        L = torch.zeros(input.size()[0])
        
        all_labels_idx = np.arange(target.size()[1])
        
        Y = float(target.size()[1])
        J = torch.nonzero(target)

        for i in range(batch_size): 
            
            msk = np.ones(target.size()[1], dtype = bool)
            
            # Find the positive label for this example
            j = J[i, 1]
            positive_indices[i, j] = 1
            msk[j] = False
            
            # initialize the sample_score_margin
            sample_score_margin = -1
            num_trials = 0
            
            neg_labels_idx = all_labels_idx[msk]

            while ((sample_score_margin < 0) and (num_trials < max_num_trials)):
                 
                #randomly sample a negative label
                neg_idx = random.sample(list(neg_labels_idx), 1)[0]
                msk[neg_idx] = False
                neg_labels_idx = all_labels_idx[msk]
                
                num_trials += 1
                # calculate the score margin 
                sample_score_margin = 1 + input[i, neg_idx] - input[i, j] 
            
            if sample_score_margin < 0:
                # checks if no violating examples have been found 
                continue
            else: 
                loss_weight = np.log(math.floor((Y-1)/(num_trials)))
                L[i] = loss_weight
                negative_indices[i, neg_idx] = 1
                
        loss = L * (1-torch.sum(positive_indices*input, dim = 1) + torch.sum(negative_indices*input, dim = 1))
        
        # ctx.save_for_backward(input, target)
        # ctx.L = L
        # ctx.positive_indices = positive_indices
        # ctx.negative_indices = negative_indices
        
        return torch.sum(loss , dim = 0, keepdim = True)

    # This function has only a single output, so it gets only one gradient 
    @staticmethod
    def backward(input, target, grad_output):
        #input, target = ctx.saved_variables
        L = Variable(torch.unsqueeze(ctx.L, 1), requires_grad = False)

        positive_indices = Variable(ctx.positive_indices, requires_grad = False) 
        negative_indices = Variable(ctx.negative_indices, requires_grad = False)
        grad_input = grad_output*L*(negative_indices - positive_indices)

        return grad_input, None, None    

      
class WARPLoss(nn.Module): 
    def __init__(self, max_num_trials = None): 
        super(WARPLoss, self).__init__()
        self.max_num_trials = max_num_trials
        
    def forward(self, input, target): 
        return WARP.apply(input, target, self.max_num_trials)

In [ ]:
a = np.load('/notebook/Relations_Learning/a200.npz.npy')
b = np.load('/notebook/Relations_Learning/b200.npz.npy')
c = np.load('/notebook/Relations_Learning/c200.npz.npy')

In [ ]:
lr_warp = 0.001
epoch =0

wp = WARP()
while (epoch < 1):
    print ("warp loss!")
    a_torch = torch.tensor(a, requires_grad=True)
    b_torch = torch.tensor(b, requires_grad=True)
    c_torch = torch.tensor(c, requires_grad=True)
    list_of_inputs = []
    list_of_targets = []
    for entity, filt in zip(test_triples, test_filter):
        p = entity[0]
        q = entity[1]
        r = entity[2]

        candidate_values = torch.sum(a_torch[p, :] * b_torch[q, :] * c_torch, axis=1)

        for obj in filt:
            idxs = (candidate_values == obj).nonzero(as_tuple=False)
            candidate_values[idxs] = 0.0
            
        candidate_values = torch.sigmoid(candidate_values)

        target = torch.zeros(len(candidate_values))
        target[r] = 1.0
        list_of_inputs.append(candidate_values)
        list_of_targets.append(target)

    inputs = torch.stack(list_of_inputs)
    print (list_of_inputs[0].shape, inputs.shape) #should be batch_size*
    targets = torch.stack(list_of_targets)
    print (list_of_targets[0].shape, targets.shape) #should be batch_size*
    
    #batch_size = 16
    #for i in range(inputs.shape[0]//batch_size):
        #print (i)
        #end = min(inputs.shape[0] - 1, (i+1)*batch_size)
    loss = wp.forward(inputs, targets) 
    print ("warp loss is counted", loss)

        #if (i ==0):
    loss.backward()
    

    a = a - lr_warp*a_torch.grad.data.numpy()
    b = b - lr_warp*b_torch.grad.data.numpy()
    c = c - lr_warp*c_torch.grad.data.numpy()
    epoch += 1

plt.xlabel("Iteration")
plt.ylabel("HR@10")
plt.title(f"FB15k-237 / GCP-GD3(R={rank}) G")
plt.plot(np.arange(1, it+1), hr_arr[:it, 2], '-*', c="#8b0a50")

In [ ]:
#@jit(nopython=True)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#@jit(nopython=True)
def hr(test_filter, test_triples, a, b, c,
       how_many=[1, 3, 10], iter_show=False, freq=3000):
    """ Calculate HR@[how_many] and MRR using filter """
    
    total = len(test_triples)
    hit = [0, 0, 0, 0]
    iteration = 0
    for entity, filt in zip(test_triples, test_filter):
        p = entity[0]
        q = entity[1]
        r = entity[2]

        candidate_values = np.sum(a[p, :] * b[q, :] * c, axis=1)
        candidate_values = sigmoid(candidate_values)
        
        top = np.argsort(candidate_values)[::-1]
        top = list(top)
        
        for obj in filt:
            top.remove(obj)
        
        ind = top.index(r)
        for i, h in enumerate(how_many):
            if ind < h:
                hit[i] += 1
        hit[3] += 1 / (1 + ind)    
        
        iteration += 1
        if iter_show:
            if iteration % freq == 0:
                print(hit[2] / iteration, hit[2], iteration)
            
    return hit[0] / total, hit[1] / total, hit[2] / total, hit[3] / total

In [ ]:
a = np.load('/notebook/wikidata_tensor/embeddings_tucker_als/embedding_size_variation/200/u0_200_237.npz.npy')
b = np.load('/notebook/wikidata_tensor/embeddings_tucker_als/embedding_size_variation/200/u1_200_237.npz.npy')
c = np.load('/notebook/wikidata_tensor/embeddings_tucker_als/embedding_size_variation/200/u2_200_237.npz.npy')

In [ ]:
%%time
shape = (100, 100, 100)
coo, vals = gen_coo_tensor(init_shape, density=0.02)
assert check_coo_tensor(coo)!= "Bad"

In [ ]:
max_iter = 20
rank = 3

In [ ]:
%%time

a, b, c, err, it = gcp_gd(
    coo, vals, shape,
    bernoulli_logit_loss,
    bernoulli_logit_loss_grad,
    rank=rank,
    lr=0.1,
    l2=0,
    max_iter=max_iter,
    tol=1e-8,
    seed=13,
    show_iter=False,
    it_over=True,
)

In [ ]:
plt.xlabel("Iteration")
plt.ylabel("Relative error")
plt.title(f"Random tensor / CP-ALS3(R={rank})")
#plt.xticks(np.arange(max_iter))
plt.yscale("log")
plt.plot(np.arange(max_iter), err[:max_iter], 'g-*')

# CP-ALS3:

In [ ]:
%%time
max_iter = 20
rank = 3
a, b, c, err, it = cp.cp_als3(coo, vals, shape, rank=rank, max_iter=max_iter)

In [ ]:
plt.xlabel("Iteration")
plt.ylabel("Relative error")
plt.title(f"Random tensor / CP-ALS3(R={rank})")
#plt.xticks(np.arange(max_iter))
plt.yscale("log")
plt.plot(np.arange(max_iter), err[:max_iter], 'g-*')